In [1]:
import spacy
import pandas as pd
import re
import jsonlines
from utils.prepare_rules import create_rules 
from utils.prepare_data import create_prepare_file
from utils.expand_model import expand_model
from tqdm import tqdm
import os.path
from spacy.language import Language
from spacy import displacy

In [2]:
DATA='data/pledges - pledges.csv'
BRANDS='data/brands.jsonl'
RULES='data/rules.jsonl'
DATA_PREPARE='data/pledges_prepare.csv'

BRAND_PATH = 'data/brands/'
RULES_PATH = 'data/rules/'

#### Загружаем данные для парсинга и данные по брендам

In [3]:
data = pd.read_csv(DATA, delimiter='|')

In [4]:
brands = list(jsonlines.open(BRANDS))
list_brands = [x['id'] for x in brands]

### Генерируем правила для моделей и марок и сохраняем их в файлы

In [5]:
rules = create_rules(brands)

# save in file
with jsonlines.open(RULES, mode='w') as writer:
    writer.write_all(rules)
    
for brand in list_brands:
    rules_model = create_rules(brands)
    try:
        models = jsonlines.open(BRAND_PATH + brand +'.jsonl')
        rules_model = create_rules(models, label='MODEL', prefix=brand+'_')
        with jsonlines.open(RULES_PATH + brand + '_rules.jsonl', mode='w') as writer:
            writer.write_all(rules_model)
    except:
         print('Для бренда '+ brand+ ' не найден файл с моделями ' + BRAND_PATH + brand +'.jsonl')

Для бренда ACURA не найден файл с моделями data/brands/ACURA.jsonl
Для бренда ADR GRUP не найден файл с моделями data/brands/ADR GRUP.jsonl
Для бренда ALLOY не найден файл с моделями data/brands/ALLOY.jsonl
Для бренда AMUR не найден файл с моделями data/brands/AMUR.jsonl
Для бренда ASIA не найден файл с моделями data/brands/ASIA.jsonl
Для бренда ATLETIC не найден файл с моделями data/brands/ATLETIC.jsonl
Для бренда AVTOMASH не найден файл с моделями data/brands/AVTOMASH.jsonl
Для бренда AYATS не найден файл с моделями data/brands/AYATS.jsonl
Для бренда BENTLEY не найден файл с моделями data/brands/BENTLEY.jsonl
Для бренда BLUMHARDT не найден файл с моделями data/brands/BLUMHARDT.jsonl
Для бренда BRILLIANCE не найден файл с моделями data/brands/BRILLIANCE.jsonl
Для бренда BSS-METACO не найден файл с моделями data/brands/BSS-METACO.jsonl
Для бренда BUNGE не найден файл с моделями data/brands/BUNGE.jsonl
Для бренда CAMC не найден файл с моделями data/brands/CAMC.jsonl
Для бренда CAT не на

### Попытаемся отбить пробелы
Возможно это не слишком хорошая идея...

In [6]:
list_models = []
for brand in list_brands:
    try:
        reader = jsonlines.open('data/brands/'+ brand +'.jsonl')
        list_models = list_models + list(reader)
    except:
        pass

In [7]:
str_with_space = create_prepare_file(data=data['vehicleproperty_description_short'].values, brands=brands+list_models)

data['with_space'] = str_with_space
data.to_csv(DATA_PREPARE, index=False)

In [8]:
data = pd.read_csv(DATA_PREPARE, delimiter=',')

### NER

In [9]:
rules = list(jsonlines.open(RULES))
for brand in list_brands:
    if os.path.exists(RULES_PATH + brand + '_rules.jsonl'):
            file = jsonlines.open(RULES_PATH + brand + '_rules.jsonl')
            rules = rules + (list(file))

In [22]:
nlp = spacy.load("ru_core_news_sm",exclude=['tok2vec', 'morphologizer', 'parser', 'senter', 'attribute_ruler', 'lemmatizer'])

config = { "overwrite_ents": True }
ruler = nlp.add_pipe("entity_ruler", before="ner", config=config) #.from_disk(RULES_PATH +"all_rules.jsonl", )
ruler.add_patterns(rules)

nlp.add_pipe("expand_model", after="ner")

brand = []
model = []
for article in tqdm(data['with_space']):
    article = str(article)
    doc = nlp(article)
    list_b = []
    list_m = []
    for ent in doc.ents:
        if ent.label_ == 'BRAND': list_b.append(ent.ent_id_)
        if ent.label_ == 'MODEL': list_m.append(ent.ent_id_)
    brand.append(None if len(list_b) == 0 else list_b)
    model.append(None if len(list_m) == 0 else list_m)

data = data.assign(Brand_NER=[x[0] if x and len(x) else None for x in brand])
data = data.assign(Brand_NER_list=[', '.join(x) if x and len(x) else None for x in brand])
data = data.assign(Model_NER=[', '.join(x) if x and len(x) else None for x in model])


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [00:23<00:00, 423.62it/s]


In [11]:
# data['Brand_NER'].value_counts()

In [23]:
data['Brand_NER'].isnull().value_counts()

False    8315
True     1685
Name: Brand_NER, dtype: int64

In [24]:
data['Model_NER'].isnull().value_counts()

True     5974
False    4026
Name: Model_NER, dtype: int64

In [25]:
# brands = list(set(data['Brand_NER'].values))
# print(brands)
brand = 'toyota'#brands[0:1]:
d = data[data["Brand_NER"]==brand][data["vehicleproperty_description_short"] != 'Автомобиль']

print('BRAND ' + brand)
print('count ' + str(d.shape[0]))
d[0:10][['with_space', 'Brand_NER','Brand_NER_list', 'Model_NER']]

BRAND toyota
count 455


/tmp/ipykernel_378631/723259441.py:4: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



,with_space,Brand_NER,Brand_NER_list,Model_NER
id,,,,
6,Toyota Land Cruiser 200легковой2012 г.вцве...,toyota,toyota,TOYOTA_land cruiser
49,ТОЙОТА Ярис 2007 г.в. ПТС - 77 ТР 013219 о...,toyota,toyota,TOYOTA_yaris
70,Автомобиль TOYOTA CAMRY легковойгос. номер...,toyota,toyota,TOYOTA_camry
74,TOYOTA CELICA ПТС 78 ТР 410668 выдан 31.05....,toyota,toyota,TOYOTA_celica
80,TOYOTA CROWN ATHLETE2004 года выпусканаиме...,toyota,toyota,TOYOTA_crown
109,автомашина TOYOTA AVENSIS 2008 года выпуска...,toyota,toyota,TOYOTA_avensis
111,TOYOTA RAV4 2007 года выпускацвет черный,toyota,toyota,TOYOTA_rav4
140,МаркаМодель TOYOTA LAND CRUISER 120 Наимен...,toyota,toyota,TOYOTA_land cruiser
149,МаркаМодель TOYOTA Camry Наименование (тип...,toyota,toyota,TOYOTA_camry


In [27]:
data.index.name= 'id'
df = data[data["Brand_NER"]==brand][data["vehicleproperty_description_short"] != 'Автомобиль']
df

/tmp/ipykernel_378631/324810696.py:2: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



,id,notificationreferencenumber,vehicleproperty_vin,vehicleproperty_description_short,nBody,nChas,BrandModel,BrandModel2,BrandModel3,Brand,Brand2,with_space,Brand_NER,Brand_NER_list,Model_NER
id,,,,,,,,,,,,,,,
6,28,2014-000-000008-548,JTMHT05J705097647,Toyota Land Cruiser 200легковой2012 г.вцвет зо...,NaN,NaN,TOYOTA LAND CRUISER PRADO 120,NaN,NaN,TOYOTA,NaN,Toyota Land Cruiser 200легковой2012 г.вцве...,toyota,toyota,TOYOTA_land cruiser
49,125,2014-000-000089-371,VNKKL98380A178355,ТОЙОТА Ярис 2007 г.в. ПТС - 77 ТР 013219 от 20...,NaN,NaN,NaN,NaN,NaN,TOYOTA,NaN,ТОЙОТА Ярис 2007 г.в. ПТС - 77 ТР 013219 о...,toyota,toyota,TOYOTA_yaris
70,157,2014-000-000130-491,JTNBE40K403180449,Автомобиль TOYOTA CAMRY легковойгос. номер Т 0...,NaN,NaN,TOYOTA CAMRY,NaN,NaN,LEXUS,NaN,Автомобиль TOYOTA CAMRY легковойгос. номер...,toyota,toyota,TOYOTA_camry
74,161,2014-000-000133-983,JTDDR32T920121452,TOYOTA CELICAПТС 78 ТР 410668 выдан 31.05.2013...,NaN,NaN,NaN,NaN,NaN,LEXUS,NaN,TOYOTA CELICA ПТС 78 ТР 410668 выдан 31.05....,toyota,toyota,TOYOTA_celica
80,167,2014-000-000139-640,NaN,TOYOTA CROWN ATHLETE2004 года выпусканаименова...,NaN,NaN,NaN,NaN,LADA 2107 CLASSIC,NaN,NaN,TOYOTA CROWN ATHLETE2004 года выпусканаиме...,toyota,toyota,TOYOTA_crown
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9947,36692,2014-000-008870-905,SB1BR56L40E147507,Модель автомобиля: TOYOTA AVENSISгод выпуска:2005,NaN,NaN,NaN,NaN,NaN,TOYOTA,NaN,Модель автомобиля: TOYOTA AVENSIS год выпус...,toyota,toyota,TOYOTA_avensis
9963,36708,2014-000-008885-737,JTDBT1238Y0084982,Модель автомобиля: TOYOTA ECHOгод выпуска:2000,NaN,NaN,NaN,NaN,NaN,LEXUS,NaN,Модель автомобиля: TOYOTA ECHO год выпуска:...,toyota,toyota,TOYOTA_echo
9972,36717,2014-000-008894-098,5TDZA22C84S164753,Модель автомобиля: TOYOTA Siennaгод выпуска:2004,NaN,NaN,NaN,NaN,NaN,TOYOTA,NaN,Модель автомобиля: TOYOTA Sienna год выпуск...,toyota,toyota,TOYOTA_siena


In [28]:
from dash import Dash, dash_table, dcc, html
from jupyter_dash import JupyterDash
from dash.dependencies import Input, Output
import pandas as pd


def generate_table(dataframe, max_rows=10):
    return html.Table([
        html.Thead(
            html.Tr([html.Th(col) for col in dataframe.columns])
        ),
        html.Tbody([
            html.Tr([
                html.Td(dataframe.iloc[i][col]) for col in dataframe.columns
            ]) for i in range(min(len(dataframe), max_rows))
        ])
    ])

app = JupyterDash(__name__)

# app.layout = html.Div([
#     html.H4(children='US Agriculture Exports (2011)'),
#     generate_table(df)
# ])
app.layout = dash_table.DataTable(
    style_data={
        'whiteSpace': 'normal',
    },
    css=[{
        'selector': '.dash-spreadsheet td div',
        'rule': '''
            line-height: 15px;
            height: auto;
            display: block;
            text-align: left;
            overflow: hidden;
        '''
    }],
    data=df.to_dict('records'),
    columns=[{"name": i, "id": i} for i in df.columns]
)

app.run_server(mode='inline')

In [69]:
# data[data["Brand_NER"].isnull()][data["vehicleproperty_description_short"] != 'Автомобиль'][1000:1050][['with_space', 'Brand_NER']]

# docs = []
# for index, text in l.iterrows():
#     doc = nlp(text['with_space'])
#     html = displacy.render(doc, style="ent", jupyter=True)

# # displacy.serve(docs, style="ent")

In [ ]:
# data[data['Brand_NER'] == 'hyundai'][data['Model_NER'].isnull()]

In [ ]:
df3 = df2[df2['Brand_NER'] == 'ford']

In [86]:
models = jsonlines.open('data/brands/FORD.jsonl')
rules_model = create_rules(models, label='MODEL')
# print(rules_model)
with jsonlines.open('data/brands/FORD_rules.jsonl', mode='w') as writer:
    writer.write_all(rules_model)

nlp = spacy.load("ru_core_news_sm")

config = { "overwrite_ents": True }
ruler = nlp.add_pipe("entity_ruler", before="ner", config=config).from_disk("data/brands/FORD_rules.jsonl", )

ents = []
for article in tqdm(df3['with_space'].values):
    article = str(article)
    doc = nlp(article)
    list_brands = [ent.ent_id_ for ent in doc.ents if ent.label_ == 'MODEL']
    ents.append(None if len(list_brands) == 0 else list_brands[0])

df3 = df3.assign(Model_NER=ents)

100%|████████████████████████| 305/305 [00:04<00:00, 71.72it/s]


In [90]:
df3[df3['Model_NER'].isnull()]

,id,notificationreferencenumber,vehicleproperty_vin,vehicleproperty_description_short,nBody,nChas,BrandModel,BrandModel2,BrandModel3,Brand,Brand2,with_space,Brand_NER,Model_NER
2201,7365,2014-000-002056-357,X9F5XXEED57K63424,FORD FOKUS2007,NaN,NaN,NaN,NaN,NaN,FORD,NaN,FORD FOKUS2007,ford,None
2998,9569,2014-000-002871-481,1FTRW14W95FA83085,Модель автомобиля: FORD F-150год выпуска:2005,NaN,NaN,NaN,NaN,NaN,FORD,NaN,Модель автомобиля: FORD F-150год выпуска:2005,ford,None
3086,9770,2014-000-002980-660,WF0MXXGBWM7D60094,Модель автомобиля: FORD GALAXYгод выпуска:2007,NaN,NaN,NaN,NaN,NaN,FORD,NaN,Модель автомобиля: FORD GALA XYгод выпуска:...,ford,None
3136,10003,2014-000-003052-800,WJMS2NSK00C105550,Грузовой тягач седельный IVICO Stralis FORD AS...,NaN,NaN,NaN,NaN,NaN,FORD,NaN,Грузовой тягач седельный IVICO Stralis FORD ...,ford,None
6113,23757,2014-000-005578-095,1FTFW1EV6AFB24193,Модель автомобиля: FORD F150год выпуска:2009,NaN,NaN,NaN,FORD CARGO,FORD CARGO,FORD,NaN,Модель автомобиля: FORD F150год выпуска:2009,ford,None
